In [49]:
import sys
sys.path.append('../')
import tqdm
from multiprocessing import Pool
from Bio.Blast.Applications import NcbiblastnCommandline
from utility.file_utility import FileUtility

ez_taxa_dict={x.split()[0]:x.split()[1].split(';') for x in FileUtility.load_list('/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/EZ/raw/eztaxon_id_taxonomy.txt')}


res=FileUtility.read_fasta_sequences_ids('../../16S_datasets/ra/markers/RA_vs_ut&h_chi2_relative.fasta')

In [50]:
res

{'1.+': ('tcagatgtcttctagagatatcattgggtattagtcttactttcgcaaggttactccccaagtggtgggcaggttggatacgcgttactcacccgtgcgccggtcgacg',
  '1.+ p-val:3.67987643055e-07'),
 '10.+': ('tcagatgtcttctagagatatcattgggtattagtcttactttcgcaaggttatccccaagtggtggg',
  '10.+ p-val:6.33292247954e-06'),
 '100.+': ('tcagatgtcttctagagacgtcattgggtattagtcttactttcgcaaggttatccccaagtgg',
  '100.+ p-val:4.66809449808e-05'),
 '101.+': ('tgctgcctcccgtaggagtttggaccgtgtctcagttccaatggtgggggaccttcctctcacgaacccctactgatcgtcgccttgg',
  '101.+ p-val:4.66809449808e-05'),
 '102.+': ('ttatccccaacgtggtgggcaggttggatacgcgttactcacccgtgcgccggtcgacg',
  '102.+ p-val:4.66809449808e-05'),
 '103.+': ('aatgtcgtttcccctcgacttgcatgtgttaagcctgtagctagcgttcatcctgagccaggatcaaactctgactgag',
  '103.+ p-val:4.66809449808e-05'),
 '104.+': ('tcagatgtcttctagagatatcattgggtattagtcttactttcgcaaggttatccccaagtggtgggcaggttggatacgcgttactcacccgtgcgccggtcgacgcccatcaaaagcaagctttcgatgtcgtttcccctcgacttgcatgtgttaagcctgtagctagcgttcatcctgagccaggatcaaactctgactgagcgggctggcaaggc

In [8]:
pattern='tcagatgtcttctagagatatcattgggtattagtcttactttcgcaaggttactccccaagtggtgggcaggttggatacgcgttactcacccgtgcgccggtcgacg'

In [26]:
FileUtility.create_fasta_file('temp.fasta',[pattern],['testseq'])

In [27]:



blastx_cline=NcbiblastnCommandline(query='temp.fasta', db="/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/EZ/raw/eztaxon_qiime_full.fasta", evalue=0.001, outfmt=5, out="opuntia.xml")

In [32]:
ez_taxa_dict['120423']

['Bacteria',
 'Bacteroidetes',
 'Bacteroidia',
 'Bacteroidales',
 'Prevotellaceae',
 'Prevotella',
 'GQ492062_s']

In [55]:
from Bio.Blast import NCBIXML

f=open("opuntia.xml",'r')
blast_records = NCBIXML.parse(f)

flag=False
score=-1
alignment_length=-1
results=[]
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if not flag and score==-1:
                score=hsp.score
                alignment_length=hsp.align_length
                flag=True
            
            if hsp.score >= score and hsp.align_length>=alignment_length:
                results.append((ez_taxa_dict[alignment.hit_id],hsp.expect))

In [64]:
def is_not_too_ambiguous(results):
    levels_id={'Superkingdom':1,'phylum':1,'class':2,'order':3,'family':4,'genus':5,'species':6}
    id_level={y:x for x,y in levels_id.items()}
    
    if len(set([x[0][levels_id['family']] for x in results]))>1:
        return False
    genuses=set([x[0][levels_id['genus']] for x in results])
    if len(genuses)==1:
        species=set([x[0][levels_id['species']] for x in results])
        if len(species)==1:
            return ';'.join(results[0][0])
        else:
            return '\makecell{' +';'.join(results[0][0][0:6])+'\\\\'+'\\\\'.join([x[0][6] for x in results])+'}'
    else:
        return '\makecell{' +';'.join(results[0][0][0:5])+'\\\\'+'\\\\'.join([x[0][5:6] for x in results])+'}'
        
    

In [65]:
';'.join(results[0][0:6])

TypeError: sequence item 0: expected str instance, list found

In [67]:
is_not_too_ambiguous(results)

'\\makecell{Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella\\\\GQ492062_s\\\\DQ797177_s\\\\HQ782802_s\\\\Prevotella_copri\\\\EU774357_s}'

In [48]:
levels_id={'Superkingdom':1,'phylum':1,'class':2,'order':3,'family':4,'genus':5,'species':6}


[';'.join(x[0]) for x in results]

[('Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella;GQ492062_s',
  1.4994e-49),
 ('Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella;DQ797177_s',
  1.4994e-49),
 ('Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella;HQ782802_s',
  1.4994e-49),
 ('Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella;Prevotella_copri',
  1.4994e-49),
 ('Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella;EU774357_s',
  1.4994e-49)]

In [19]:
'tcagatgtcttctagagatatcattgggtattagtcttactttcgcaaggttactccccaagtggtgggcaggttggatacgcgttactcacccgtgcgccggtcgacg' in 'catgcaagtcgaacgagggaatattcttcggaatataaatcgagtggcgaaagggtgagtaatgcatagatgacatacctttaggataagaatagctttgtgaaaacggagttaatgcttaatgattacgaaaagaaataaattcctttcgtcaaagggggggcagcaatgcctcttgcctagagagtggtttatgtcctatcagctagttggtggggtaaaggcctaccaaggcttagacgggtagccggccttagagggtggtcggccacactgggactgagacactgcccagactcctacgggaggctgcagtcgagaatcattcgcaatgcccgaaagggtgacgatgcgacgccgcgtgagcgaagaaggccctagggttgtaaagctctgtcgggggttaagaaatgcaggttggttaatacccagcctgtttgacaaaggccccaaaggaagccacggctaactctgtgccagcagccgcggtaatacagaggtggcaagcgttgttcggtattattgggcgtaaagagcacgtaggtggttttgtaagtcagatgtgaaagccttctgttcaaaggaagaacggcatctgaaactgcaaatcttgagtgtaggaggggagagtggaacttctggtggagcggtgaaatgcgtagatatcagaaggaacgtcggcggcgaaagcgactctctggcctattactgacactgagtgtgcgaaagttaggggagcaaacgggattagataccccggtagtcctaactgtaaacgatgggcactaagtagagggaattaccaatgttctctctgccgtagctaacgcattaagtgccccgcctggggagtacggccgcaaggctaaaactcaaaagaattgacgggggctcgcacaagcggtggagcatgtggcttaattcgatgcaacgcgaaaaaccttaccggggtttgacatggtagaagtaggagcccgaaagggtaaccaacggtatccagtccgtaactactgcaggtgttgcatggctgtcgtcagctcgtgtcgtgagacgttgggttaagtcccctaacgagcgaaacccttatttttagttaccatcgggttatgccgggcactctagaaagactgccgtcgtcaagacggaggaaggtggggatgacgtcaagtcatcatggcccttataccccgggctgcgcacgtgctacaatggtcggtacaaaaggaagcaagaccgagaggtggagcgaatcctataaagccgatctcagttcggattggaggctgaaattcgcctccatgaagttggaatcgctagtaatcgcggatcatcatcgccgcggtgaatatgttcccgagctcagatgtcttctagagatatcattgggtattagtcttactttcgcaaggttactccccaagtggtgggcaggttggatacgcgttactcacccgtgcgccggtcgacgcttgtacacaccgcccgtcaagccacccaagtaagatgcacccgaaaacatttatctaaccacttgtgggggaaaatgttgaaggagtgttttgtga'

True